In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import time
import os
from os import path
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from scipy.linalg import sqrtm

In [6]:
movie = pd.read_csv('/content/movies.csv', sep=',',names=['MovieID','Title','Genres'],encoding='latin-1')
rating = pd.read_csv('ratings.csv',sep=',', names=['UserID','MovieID','Rating','Timestamp'])
user = pd.read_csv('users.csv',sep=',',names=['Gender','Age','Occupation','Zip-code'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/movies.csv'

In [4]:
rating

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
Original_ratings = rating.pivot_table(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
Original_ratings

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,4.0,3.0,0.0,4.0,3.0,4.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Create Singular value decomposition
def svd(k):                                           
    rating_matrix = np.array(Original_ratings)        #Convert Datafram into matrix
    mean = np.mean(rating_matrix, axis=0)             
    rating_matrix = rating_matrix + mean              #Changing 0 to higher magnitude values
    U, sigma, V=np.linalg.svd(rating_matrix, full_matrices=False)
    sigma=np.diag(sigma)
    # we take only the k most significant features
    sigma=sigma[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
    sigmat=sqrtm(sigma)           #Change matrix to square root matrix
    Usk=np.dot(U,sigma)           #dot product of U and sigma matrices
    skV=np.dot(sigma,V)           #dot product of sigma and V matrices
    UsV = np.dot(Usk, skV)        #final svd matrices UsV
    UsV = UsV - mean
    UsV = pd.DataFrame(UsV,columns=Original_ratings.columns,index=Original_ratings.index)       #Matrix to dataframe
    return UsV

In [4]:
predicted_ratings = svd(24)

NameError: name 'np' is not defined

In [12]:
predicted_ratings

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,7316.921390,1915.658572,1105.428176,376.494243,749.429370,2632.624171,1265.835024,210.271166,185.610518,2356.180428,3119.911471,299.899845,357.743566,442.726141,277.241368,1984.034979,2710.486567,404.052221,739.662790,295.071785,3566.638378,988.457458,281.003783,1507.314007,2764.942197,320.403400,173.879191,553.188013,1197.678333,195.687783,391.102222,4511.545003,8.390977,5644.193347,178.435650,2999.124139,22.935118,71.528335,3856.372333,98.657353,...,79.659826,183.894242,320.749586,1377.959627,622.850880,330.001995,137.865336,173.067996,87.834447,64.499457,178.290153,52.968448,315.013092,439.470846,924.664742,530.712023,415.115779,503.223355,36.451386,600.539618,24.545107,50.368918,45.694536,297.575488,285.777359,36.540481,34.647215,18.831271,35.689394,37.323718,217.001746,16.374188,59.403019,158.501615,113.772253,2377.658885,944.653666,159.043522,108.624462,1106.495351
2,8374.192159,2354.831321,1475.159059,505.777909,941.679920,4014.112433,1658.051652,205.102529,347.631123,3687.687570,4172.874298,368.705813,301.977830,638.333438,426.612610,2835.390866,3566.409997,540.112073,987.963178,522.932251,5274.827781,1441.964683,435.674084,1944.374099,3822.285138,411.590458,198.404634,740.247482,1467.082284,290.626717,528.625166,5905.705583,14.679116,6676.134891,261.705626,3981.259928,26.310057,68.061339,4898.247363,145.938130,...,80.925163,220.347630,422.010688,1768.354793,815.996243,433.703257,184.933334,218.523079,105.073502,83.795365,263.411045,71.139416,426.220033,618.636027,1237.594778,675.617458,572.391972,692.722509,47.488481,817.094886,35.791062,80.722346,70.607588,411.309970,392.579607,45.212596,39.996300,22.183660,40.254350,45.233587,268.090806,15.695923,55.785815,229.610633,184.092123,2975.313486,1176.477097,202.737804,130.238196,1421.029461
3,6830.518905,1956.587924,1141.640038,349.300469,720.452316,2903.939848,1200.088113,176.582526,223.767737,2687.630352,2989.762428,318.292995,249.837628,417.774588,311.233765,1993.082037,2421.104910,442.422516,778.373392,336.377273,3977.377475,975.002811,290.933034,1493.509152,2672.393997,285.377762,131.476281,501.707644,1266.680736,190.218601,344.025006,4497.261230,8.615337,5186.856427,159.061311,2748.548228,19.531799,61.674599,3823.159456,83.673481,...,74.712357,164.828586,317.782793,1298.813558,667.744215,353.003096,147.730796,187.272657,86.939622,73.337081,179.898523,55.581641,346.617576,444.745362,923.012488,538.454460,425.394680,527.563839,36.143058,616.455880,25.781099,53.710910,48.782173,306.587498,304.178716,40.061293,38.754070,19.243238,36.358858,38.953952,216.001893,13.807613,52.823133,174.744411,137.728817,2381.764051,936.069695,159.718917,105.481305,1066.376881
4,5481.363484,1494.746611,897.308211,308.433704,583.459338,2553.241683,998.766834,137.574873,188.878777,2174.168185,2433.512142,257.719476,213.515359,379.183768,245.397649,1721.966021,2058.849068,359.350748,624.493509,286.547486,3285.016590,842.536026,239.810682,1218.746104,2320.633061,246.548865,120.203889,447.860323,1105.112847,184.008702,297.383658,3933.794481,8.364478,4202.272820,160.444838,2327.472791,17.206587,49.549201,3074.977620,78.558821,...,60.329945,147.595742,280.135629,1069.520127,577.309360,311.235919,131.059110,165.930529,80.429554,63.936947,172.269672,52.125436,287.837205,393.541159,830.673507,452.168602,357.688117,482.489440,35.719758,581.210949,24.931156,52.586210,51.371450,261.183303,262.284806,33.719911,33.461624,17.082462,31.027652,32.432998,190.335345,11.833215,39.223023,139.028463,121.493606,1925.960535,824.563304,142.178452,95.510568,910.407793
5,8265.547042,2161.675262,1314.402880,529.3

In [13]:
def recommend(n,userid):
  sorted_predictions = predicted_ratings.iloc[userid-1].sort_values(ascending=False) #UserID starts at 1, not 0
  print('Recommending the highest {0} predicted ratings movies for you.'.format(n))
  recommendations = pd.merge(movie, pd.DataFrame(sorted_predictions),how = 'inner',left_on='MovieID',right_on='MovieID').rename(columns={userid:'rank'}).sort_values('rank',ascending=False).head(n)
  return recommendations

In [14]:
recommend(10,1)

Recommending the highest 10 predicted ratings movies for you.


,MovieID,Title,Genres,rank
2651,2858,American Beauty (1999),Comedy|Drama,10461.326220
253,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,9904.305073
1106,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,9649.469444
1848,2028,Saving Private Ryan (1998),Action|Drama|War,8602.544186
1120,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,8562.603872
579,593,Silence of the Lambs,The (1991),8551.307083
1108,1198,Raiders of the Lost Ark (1981),Action|Adventure,8526.987665
2374,2571,Matrix,The (1999),8233.158380
2557,2762,Sixth Sense,The (1999),8152.675025
513,527,Schindler's List (1993),Drama|War,8074.693716
